In [2]:
import json
import random
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import tkinter as tk
from tkinter import END, messagebox

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents file
data_file = open(r"C:\Users\R Sobha Supriya\Downloads\intents1.json").read()
intents = json.loads(data_file)

# Process intents
words = []
classes = []
documents = []
ignore_words = ['?', '!']

# User preferences with login counts
user_preferences = {
    'sobha': {'name': 'Sobha', 'likes': ['health'], 'login_count': 0},
    'priya': {'name': 'Priya', 'likes': ['education'], 'login_count': 0},
    'krishna': {'name': 'Krishna', 'likes': ['entertainment'], 'login_count': 0},
    'sailaja': {'name': 'Sailaja', 'likes': ['medical'], 'login_count': 0},
}

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes:", classes)
print(len(words), "unique lemmatized words:", words)

# Save words and classes to pickle files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append((bag, output_row))

# Shuffle and convert to numpy arrays
random.shuffle(training)
train_x = np.array([i[0] for i in training])
train_y = np.array([i[1] for i in training])

print("Training data created")

# Create model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit model and save
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created")

# Load the model
model = keras.models.load_model('chatbot_model.h5')

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Additional responses
additional_responses = {
    "facts": [
        "Did you know honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3000 years old!",
        "Bananas are berries, but strawberries aren't!",
        "Octopuses have three hearts.",
        "A day on Venus is longer than a year on Venus."
    ],
    "jokes": [
        "Why don't scientists trust atoms? Because they make up everything!",
        "Why did the scarecrow win an award? Because he was outstanding in his field!",
        "Why did the bicycle fall over? Because it was two-tired!"
    ],
    "riddles": [
        "What has keys but can't open locks? A piano.",
        "I speak without a mouth and hear without ears. I have no body, but I come alive with the wind. What am I? An echo.",
        "What can travel around the world while staying in a corner? A stamp."
    ]
}

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    return [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json, username):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    user_info = user_preferences.get(username, {})
    
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break

    if tag in additional_responses:
        result += "\n" + random.choice(additional_responses[tag])

    # Only greet on the first interaction
    if user_info['login_count'] == 1:
        return f"Hey {user_info['name']}, {result}"
    
    return result

def chatbot_response(msg, username):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents, username)
    return res

# GUI with tkinter
current_username = None

# GUI setup
base = tk.Tk()
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=False, height=False)

# Create Chat window
ChatLog = tk.Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=tk.DISABLED)

# Bind scrollbar to Chat window
scrollbar = tk.Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Color tags
ChatLog.tag_config("user", foreground="#0000FF")  # User messages in blue
ChatLog.tag_config("bot", foreground="#FF4500")  # Bot messages in orange

# Create Button to send message
def send():
    global current_username
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    # Check if the username is provided
    if current_username is None:
        ChatLog.config(state=tk.NORMAL)
        ChatLog.insert(END, "Bot: Please enter your username.\n\n", "bot")
        ChatLog.config(state=tk.DISABLED)
        return

    if msg != '':
        ChatLog.config(state=tk.NORMAL)
        ChatLog.insert(END, f"You: {msg}\n\n", "user")
        ChatLog.config(state=tk.DISABLED)

        res = chatbot_response(msg, current_username)
        ChatLog.config(state=tk.NORMAL)
        ChatLog.insert(END, f"Bot: {res}\n\n", "bot")
        ChatLog.config(state=tk.DISABLED)
        ChatLog.yview(END)

SendButton = tk.Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height="5",
                       bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff',
                       command=send)

# Create the box to enter message
EntryBox = tk.Text(base, bd=0, bg="white", width=29, height=5, font="Arial")

# Place all components on the screen
scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

# Hide the main chat window initially
base.withdraw()

def set_username():
    global current_username
    username = username_entry.get().strip().lower()
    if username in user_preferences:
        current_username = username
        user_info = user_preferences[username]
        user_info['login_count'] += 1

        ChatLog.config(state=tk.NORMAL)
        ChatLog.insert(END, f"Bot: Hey {user_info['name']}, this is your {user_info['login_count']} time logging in!\n\n", "bot")
        ChatLog.config(state=tk.DISABLED)

        username_window.destroy()
        base.deiconify()
    else:
        messagebox.showerror("Error", "Username not recognized. Please Create an Account to Continue")

# Username window setup
username_window = tk.Toplevel(base)
username_window.title("Enter Username")
username_window.geometry("300x100")

username_label = tk.Label(username_window, text="Username:")
username_label.pack(pady=10)

username_entry = tk.Entry(username_window)
username_entry.pack(pady=10)

username_button = tk.Button(username_window, text="Submit", command=set_username)
username_button.pack(pady=10)

base.mainloop()


70 documents
16 classes: ['diet_advice', 'education', 'entertainment', 'goodbye', 'greeting', 'interesting_facts', 'job_roles', 'jokes', 'life_advice', 'medical_conditions', 'movie_suggestions', 'nearest_hospital', 'options', 'riddles', 'thanks', 'theatre_search']
117 unique lemmatized words: ["'s", ',', 'a', 'about', 'advice', 'an', 'any', 'anyone', 'are', 'available', 'awesome', 'be', 'best', 'blood', 'bye', 'can', 'chatting', 'cinema', 'closest', 'could', 'course', 'day', 'demand', 'diabetes', 'diet', 'do', 'domain', 'eating', 'education', 'facility', 'fact', 'film', 'find', 'for', 'fun', 'funny', 'get', 'give', 'good', 'goodbye', 'guidance', 'have', 'healthcare', 'healthy', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hospital', 'how', 'i', 'in', 'industry', 'information', 'interesting', 'is', 'job', 'joke', 'know', 'later', 'laugh', 'level', 'life', 'list', 'locate', 'love', 'make', 'manage', 'me', 'movie', 'near', 'nearby', 'nearest', 'next', 'nice', 'now', 'nutri